## Notebook 02: Tiền xử lý dữ liệu (từ kết quả EDA)

### Quy trình tổng quát
1. **Nạp dữ liệu thô**
2. **Preprocessing**: Kiểm tra
    - Dữ liệu có thiếu không?
    - Các cột chính có hợp lệ không?
    - Kiểm tra outliers

### Ghi chú từ EDA
- Phân phối rating lệch nhưng vẫn nằm hoàn toàn trong [1, 5] và không có giá trị nhiễu.
- 14.7% đánh giá 1-2 sao mang ý nghĩa tiêu cực hợp lệ => giữ nguyên

In [2]:
import sys
import os
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


from data_preprocessing import (
    load_data,
    check_missing_values,
    fill_missing_values,
    validate_data,
    detect_outliers_iqr,
    save_processed_data
)

# Config
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.set_printoptions(precision=3, suppress=True)


## 1: Load raw data

In [3]:
# Đường dẫn dữ liệu
RAW_DATA_PATH = '../data/raw/ratings_Beauty.csv'
PROCESSED_DATA_PATH = '../data/processed/ratings_Beauty_processed'

print("=" * 70)
print("BƯỚC 1: NẠP DỮ LIỆU THÔ")
print("=" * 70)
print(f"\nĐang đọc: {RAW_DATA_PATH}")
print("(Notebook 01 xác nhận dữ liệu sạch nên có thể đọc trực tiếp)")

data = load_data(RAW_DATA_PATH)

print("\n Đọc dữ liệu thành công!")
print(f"  Kích thước: {data.shape}")
print(f"  Kiểu dữ liệu: {data.dtype}")

print("\n5 dòng đầu tiên:")
for i in range(min(5, len(data))):
    print(f"  {data[i]}")

BƯỚC 1: NẠP DỮ LIỆU THÔ

Đang đọc: ../data/raw/ratings_Beauty.csv
(Notebook 01 xác nhận dữ liệu sạch nên có thể đọc trực tiếp)
 Đọc dữ liệu thành công từ ../data/raw/ratings_Beauty.csv
  Số dòng: 2,023,070

 Đọc dữ liệu thành công!
  Kích thước: (2023070,)
  Kiểu dữ liệu: [('UserId', '<U50'), ('ProductId', '<U50'), ('Rating', '<f4'), ('Timestamp', '<i8')]

5 dòng đầu tiên:
  ('A39HTATAQ9V7YF', '0205616461', 5., 1369699200)
  ('A3JM6GV9MNOF9X', '0558925278', 3., 1355443200)
  ('A1Z513UWSAAO0F', '0558925278', 5., 1404691200)
  ('A1WMRR494NWEWV', '0733001998', 4., 1382572800)
  ('A3IAAVS479H7M7', '0737104473', 1., 1274227200)


## 2: Kiểm tra tính hợp lệ dữ liệu (Data Validation)

In [4]:
print("=" * 70)
print("BƯỚC 2: KIỂM TRA TÍNH HỢP LỆ DỮ LIỆU")
print("=" * 70)

users = data['UserId']
products = data['ProductId']
ratings = data['Rating']
timestamps = data['Timestamp']

# 2.1: Completeness - Kiểm tra missing values
print("\n[2.1] KIỂM TRA GIÁ TRỊ THIẾU")
print("-" * 70)
missing_users = np.sum(users == '')
missing_products = np.sum(products == '')
missing_ratings = np.sum(np.isnan(ratings))
missing_timestamps = np.sum(timestamps == 0)

print(f"UserId thiếu:    {missing_users:,} ({missing_users/len(data)*100:.2f}%)")
print(f"ProductId thiếu: {missing_products:,} ({missing_products/len(data)*100:.2f}%)")
print(f"Rating thiếu:    {missing_ratings:,} ({missing_ratings/len(data)*100:.2f}%)")
print(f"Timestamp = 0:   {missing_timestamps:,} ({missing_timestamps/len(data)*100:.2f}%)")

if missing_users + missing_products + missing_ratings + missing_timestamps == 0:
    print("\n PASS: Dữ liệu hoàn chỉnh 100%")
    
# 2.2: Validity - Kiểm tra khoảng giá trị
print("\n[2.2] KIỂM TRA KHOẢNG GIÁ TRỊ HỢP LỆ")
print("-" * 70)
invalid_ratings = np.sum((ratings < 1) | (ratings > 5))
print(f"Rating ngoài [1,5]: {invalid_ratings:,}")

unique_ratings, counts = np.unique(ratings, return_counts=True)
print("\nPhân phối rating:") 
for val, cnt in zip(unique_ratings, counts):
    print(f"  Rating {val:.1f}: {cnt:,} ({cnt/len(ratings)*100:.1f}%)")
    
if invalid_ratings == 0:
    print("\n PASS: Tất cả ratings hợp lệ [1-5]")

# 2.3: Uniqueness - Kiểm tra trùng lặp
print("\n[2.3] KIỂM TRA TRÙNG LẶP")
print("-" * 70)
unique_users = len(np.unique(users))
unique_products = len(np.unique(products))
print(f"Users duy nhất: {unique_users:,}")
print(f"Products duy nhất: {unique_products:,}")
print(f"Sparsity: {100*(1 - len(data)/(unique_users*unique_products)):.2f}%")

user_product_pairs = list(zip(users, products))
unique_pairs = len(set(user_product_pairs))
duplicate_pairs = len(data) - unique_pairs
print(f"\nCặp (user,product) trùng: {duplicate_pairs:,} ({duplicate_pairs/len(data)*100:.2f}%)")
if duplicate_pairs > 0:
    print(" Giữ nguyên: temporal reviews hợp lệ")

BƯỚC 2: KIỂM TRA TÍNH HỢP LỆ DỮ LIỆU

[2.1] KIỂM TRA GIÁ TRỊ THIẾU
----------------------------------------------------------------------
UserId thiếu:    0 (0.00%)
ProductId thiếu: 0 (0.00%)
Rating thiếu:    0 (0.00%)
Timestamp = 0:   0 (0.00%)

 PASS: Dữ liệu hoàn chỉnh 100%

[2.2] KIỂM TRA KHOẢNG GIÁ TRỊ HỢP LỆ
----------------------------------------------------------------------
Rating ngoài [1,5]: 0

Phân phối rating:
  Rating 1.0: 183,784 (9.1%)
  Rating 2.0: 113,034 (5.6%)
  Rating 3.0: 169,791 (8.4%)
  Rating 4.0: 307,740 (15.2%)
  Rating 5.0: 1,248,721 (61.7%)

 PASS: Tất cả ratings hợp lệ [1-5]

[2.3] KIỂM TRA TRÙNG LẶP
----------------------------------------------------------------------
Users duy nhất: 1,210,271
Products duy nhất: 249,274
Sparsity: 100.00%

Cặp (user,product) trùng: 0 (0.00%)


In [5]:
print("=" * 70)
print("BƯỚC 3: PHÁT HIỆN GIÁ TRỊ BẤT THƯỜNG")
print("=" * 70)

# User behavior outliers
print("\n[3.1] USER BEHAVIOR OUTLIERS")
print("-" * 70)
user_counts = Counter(users)
ratings_per_user = np.array(list(user_counts.values()))

print(f"Ratings/user - Min: {ratings_per_user.min()}, Max: {ratings_per_user.max():,}")
print(f"Ratings/user - Mean: {ratings_per_user.mean():.1f}, Median: {np.median(ratings_per_user):.0f}")

# IQR method
q1, q3 = np.percentile(ratings_per_user, [25, 75])
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
outlier_users = np.sum(ratings_per_user > upper_bound)
print(f"\nOutlier threshold (IQR): {upper_bound:.0f}")
print(f"Users vượt ngưỡng: {outlier_users:,} ({outlier_users/len(user_counts)*100:.1f}%)")
print("\n Giữ nguyên: Power users có giá trị cho CF")

# Product popularity outliers
print("\n[3.2] PRODUCT POPULARITY OUTLIERS")
print("-" * 70)
product_counts = Counter(products)
ratings_per_product = np.array(list(product_counts.values()))

print(f"Ratings/product - Min: {ratings_per_product.min()}, Max: {ratings_per_product.max():,}")
print(f"Ratings/product - Mean: {ratings_per_product.mean():.1f}, Median: {np.median(ratings_per_product):.0f}")

q1, q3 = np.percentile(ratings_per_product, [25, 75])
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
outlier_products = np.sum(ratings_per_product > upper_bound)
print(f"\nOutlier threshold (IQR): {upper_bound:.0f}")
print(f"Products vượt ngưỡng: {outlier_products:,} ({outlier_products/len(product_counts)*100:.1f}%)")
print("\n Giữ nguyên: Popular products có giá trị cho recommendation")

BƯỚC 3: PHÁT HIỆN GIÁ TRỊ BẤT THƯỜNG

[3.1] USER BEHAVIOR OUTLIERS
----------------------------------------------------------------------
Ratings/user - Min: 1, Max: 389
Ratings/user - Mean: 1.7, Median: 1

Outlier threshold (IQR): 4
Users vượt ngưỡng: 82,659 (6.8%)

 Giữ nguyên: Power users có giá trị cho CF

[3.2] PRODUCT POPULARITY OUTLIERS
----------------------------------------------------------------------
Ratings/product - Min: 1, Max: 7,533
Ratings/product - Mean: 8.1, Median: 2

Outlier threshold (IQR): 11
Products vượt ngưỡng: 31,337 (12.6%)

 Giữ nguyên: Popular products có giá trị cho recommendation


**Nhận xét**: 
- Dataset này khá sạch và không có nhiều features (chỉ có 2 numerical features là Timestamp và Rating)
- Vì rating nằm trong đoạn [1-5] là xử lý được nên không cần thiết minmax hay log scaling

## 4: Lưu dữ liệu sạch ra `.npz`

Sau khi kiểm tra hợp lệ và outlier, chúng ta lưu toàn bộ dataset vào định dạng `.npz` để tái sử dụng ở bước modeling.


In [6]:
print("=" * 70)
print("BƯỚC 4: LƯU DỮ LIỆU SẠCH")
print("=" * 70)

CLEAN_OUTPUT_PATH = '../data/processed/ratings_Beauty_processed_clean'

# Dùng lại hàm validate_data để đảm bảo dữ liệu hợp lệ
clean_data, removed = validate_data(data)
print(f"\nSố dòng bị loại bỏ: {removed:,}")
print(f"Kích thước dữ liệu sạch: {len(clean_data):,}")

save_processed_data(clean_data, CLEAN_OUTPUT_PATH)
print(f"\nFile đã sẵn sàng: {CLEAN_OUTPUT_PATH}.npz")


BƯỚC 4: LƯU DỮ LIỆU SẠCH

=== KIỂM TRA TÍNH HỢP LỆ ===
  Số dòng ban đầu: 2,023,070
  Số dòng không hợp lệ: 0
  Số dòng còn lại: 2,023,070
  Tỷ lệ giữ lại: 100.00%

Số dòng bị loại bỏ: 0
Kích thước dữ liệu sạch: 2,023,070

=== LƯU DỮ LIỆU ===
  Saved: ../data/processed/ratings_Beauty_processed_clean.npz (compressed)
  Kích thước: 2,023,070 records
  File size: 52.29 MB

File đã sẵn sàng: ../data/processed/ratings_Beauty_processed_clean.npz
